In [1]:
import pandas as pd
import numpy as np
import lifelines as lil
from lifelines import KaplanMeierFitter
#from lifelines.datasets import load_waltons
import os
import sys
#import math
#import struct
#import pandas as pd
#import numpy as np
#import seaborn as sns
#import matplotlib.pyplot as plt
#import numpy as np
from bokeh.plotting import ColumnDataSource
from bokeh.models import HoverTool
from bokeh.io import push_notebook, show, output_notebook
from bokeh.layouts import row
from bokeh.plotting import figure
from bokeh.models import Range1d
output_notebook()
#from bokeh.plotting import figure, show#, output_file

Loading BokehJS ...

In [30]:
TEST = True
DATA_DIR = "data/"
DATA_FOLDERS = ["2014","2015","data_Q1_2016","data_Q2_2016","data_Q3_2016","data_Q4_2016"]
#DATA_FOLDERS = ["2014","2015"]
#THE_TIME = strftime("%Y-%m-%d-%H-%S", locatime())
SUMMARY_DIR = "summary_data/"
PERCENT_TOTAL_REQ = .05
FAILURE_RATE_REQ = 5
MIN_NUMBER_REQ = 100
INPUT_DIR = "survival_data"

In [31]:
target_models = []
for df in DATA_FOLDERS:
    print(SUMMARY_DIR + df + '.csv')
    summary_dats = pd.read_csv(SUMMARY_DIR + df + '.csv', header=0, nrows=200)
    #print(temp.head)
    summary_dats = summary_dats.sort_values(by="percent_total", ascending=False)
    clipped1 = summary_dats[summary_dats['percent_total'] >= PERCENT_TOTAL_REQ]
    clipped2 = summary_dats[(summary_dats['failure_rate'] >= FAILURE_RATE_REQ) & (summary_dats['drive_count'] >= MIN_NUMBER_REQ)]
    [target_models.append(m) for m in clipped1['model']]
    [target_models.append(m) for m in clipped2['model']]

    
#print(np.unique(target_models))

unique_target_models = np.unique(target_models)
print(unique_target_models)

summary_data/2014.csv
summary_data/2015.csv
summary_data/data_Q1_2016.csv
summary_data/data_Q2_2016.csv
summary_data/data_Q3_2016.csv
summary_data/data_Q4_2016.csv
['HGST HMS5C4040ALE640' 'HGST HMS5C4040BLE640' 'Hitachi HDS5C3030ALA630'
 'Hitachi HDS5C4040ALE630' 'Hitachi HDS722020ALA330' 'ST3000DM001'
 'ST4000DM000' 'ST8000DM002']


In [36]:
survival_data = []
km_data = []
for model in unique_target_models:
    model_ns = model.replace(" ", "_")
    pf = INPUT_DIR + "/survival_" +  model_ns + '.csv'# + "data_Q2_2016.csv"
    print(pf)
    tmp = pd.read_csv(pf, header=0)
    yr = 365. * 24.
    tmp['runtime_max'] = tmp['runtime_max']/yr
    tmp['uptime'] = tmp['uptime']/yr
    tmp['runtime_min'] = tmp['runtime_min']/yr
    survival_data.append(tmp)
    km = lil.KaplanMeierFitter()
    try:
        km.fit(durations=tmp['runtime_max'], event_observed=tmp['failure'], entry=tmp['runtime_min'])
        km_data.append(km)
    except:
        print "Probably too few early truncation times and too many events. Try BreslowFlemingHarringtonFitter?"
        print "Skipping ..."
        pass
    
#OUTPUT_DIR = "web/data"
#import pickle
#pickle.dump( km_data, open( OUTPUT_DIR + "/survival.p", "wb" ) )
    #km_outf = INPUT_DIR + "/survival_" +  model_ns + '.csv'# + "data_Q2_2016.csv
    #print km_outf
    #OUTPUT_DIR = "web/data"
    #survives.to_csv(OUTPUT_DIR + "/kmfit_" + model_ns + ".csv", index = False)

survival_data/survival_HGST_HMS5C4040ALE640.csv
survival_data/survival_HGST_HMS5C4040BLE640.csv
survival_data/survival_Hitachi_HDS5C3030ALA630.csv
survival_data/survival_Hitachi_HDS5C4040ALE630.csv
survival_data/survival_Hitachi_HDS722020ALA330.csv
survival_data/survival_ST3000DM001.csv
survival_data/survival_ST4000DM000.csv
survival_data/survival_ST8000DM002.csv


PicklingError: Can't pickle <function plot at 0x7f1bcbc349b0>: it's not found as lifelines.plotting.plot

In [37]:
#OUTPUT_DIR = "web/data"
#import pickle
#pickle.dump( km_data, open( OUTPUT_DIR + "/survival.p", "wb" ) )
print type(km_data)

<type 'list'>


In [38]:
km = sa.KaplanMeierFitter()
km.fit(durations=survival['runtime_max'], event_observed=survival['failure'], entry=survival['runtime_min'])
S = km.survival_function_

  serial_number  n_obs  failure  runtime_min  runtime_max  uptime
0      S3000FZ5    182        0       8551.0      12895.0  4344.0
1      S3000NSV    182        0       8456.0      12799.0  4343.0
2      S3000QAP    182        0       8648.0      12991.0  4343.0
3      S30015PW    182        0       5582.0       9925.0  4343.0
4      S3001FPA    182        0       2221.0       6564.0  4343.0


In [22]:
#import inspect
#ob = kmf
#[method for method in dir(ob)]# if callable(getattr(ob, method))]

['_KaplanMeierFitter__estimate',
 '__class__',
 '__delattr__',
 '__dict__',
 '__doc__',
 '__format__',
 '__getattribute__',
 '__hash__',
 '__init__',
 '__module__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_additive_f',
 '_additive_var',
 '_bounds',
 '_conditional_time_to_event_',
 '_divide',
 '_label',
 '_plot_estimate',
 '_predict',
 '_subtract',
 'alpha',
 'conditional_time_to_event_',
 'confidence_interval_',
 'divide',
 'durations',
 'entry',
 'event_observed',
 'event_table',
 'fit',
 'median_',
 'plot',
 'plot_loglogs',
 'plot_survival_function_',
 'predict',
 'subtract',
 'survival_function_',
 'timeline']

In [23]:
s = km.survival_function_
surv = s['KM_estimate'].values
ci = km.confidence_interval_
#print s.head()
#print s.columns
#print ci.columns
time = s['KM_estimate'].index
surv = s['KM_estimate'].values
surv_upper = ci['KM_estimate_upper_0.95'].values
surv_lower = ci['KM_estimate_lower_0.95'].values

In [34]:

def km_bokeh_plot(kms, models):
    surv_plt1 = figure(title="Survival Analysis", tools=['hover,box_zoom,wheel_zoom,save,reset'])
    hover = surv_plt1.select(dict(type=HoverTool))
    hover.tooltips = [
        ("Model ", "@model"),
        ("Time ", "@timeline"),
        ("survival fraction ", "@km_surv"),
        ("upper 95% bound ", "@surv_upper"),
        ("lower 95% bound ", "@surv_lower")
        ]
    if len(kms) > 1:
        hover.mode = 'mouse'
    else: 
        hover.mode = 'vline'
    colors = ['#1a1334', '#03c383', '#fbbf45', '#ed0345' ,  '#26294a', '#aad962', '#01545a', '#ef6a32','#017351',
              '#a12a5e', '#710162', '#110141']
    n = 0
    for km in kms:
        s = km.survival_function_
        ci = km.confidence_interval_
        time = s['KM_estimate'].index
        surv = s['KM_estimate'].values
        surv_upper = ci['KM_estimate_upper_0.95'].values
        surv_lower = ci['KM_estimate_lower_0.95'].values
        band_x = np.append(time, time[::-1])
        band_y = np.append(surv_upper, surv_lower[::-1])
        source = ColumnDataSource(
            data=dict(
                timeline=[i for i in time],
                km_surv=[i for i in surv],
                model=[models[n] for i in time],
                surv_lower=[i for i in surv_lower],
                surv_upper=[i for i in surv_upper]
            )
        )
        surv_plt1.patch(band_x, band_y, color=colors[n], fill_alpha=0.2)
        surv_plt1.line('timeline', 'km_surv', line_width = 2, alpha=.8, source = source, legend=models[n], color=colors[n])
        n += 1
    surv_plt1.xaxis.axis_label = 'Time (Years)'
    surv_plt1.yaxis.axis_label = 'Kaplan-Meier Estimation (survival fraction)'
    
    # grid styles
    #surv_plt1.grid.grid_line_alpha = 0.4
    surv_plt1.grid.grid_line_alpha = 0
    surv_plt1.ygrid.band_fill_color = "grey"
    surv_plt1.ygrid.band_fill_alpha = 0.2
    surv_plt1.x_range.range_padding = 0
    surv_plt1.legend.location = "bottom_left"
    surv_plt1.plot_height = 500
    surv_plt1.plot_width = 700
    surv_plt1.border_fill_color = "black"
    surv_plt1.background_fill_color = "white"
    surv_plt1.min_border_left = 80
    surv_plt1.outline_line_width = 1
    surv_plt1.outline_line_alpha = 0.3
    surv_plt1.outline_line_color = "white"
    surv_plt1.xaxis.axis_label_text_color="white"
    surv_plt1.yaxis.axis_label_text_color="white"
    surv_plt1.xaxis.major_tick_line_color = 'white'
    surv_plt1.yaxis.major_tick_line_color = 'white'
    surv_plt1.xaxis.minor_tick_line_color = '#C1C1C1'
    surv_plt1.yaxis.minor_tick_line_color = '#C1C1C1'
    surv_plt1.xaxis.major_label_text_color = 'white'
    surv_plt1.yaxis.major_label_text_color = 'white'
    surv_plt1.title.text_color = 'white'

    surv_plt1.y_range = Range1d(0.0, 1.02)
    #surv_plt1.x_range = Range1d(0.4, 1.1)
    #upper and lower 95% confidence intervals shown
    show(surv_plt1)
         
model = 'dne'

#km_bokeh_plot([km], ['alpha'])

km_bokeh_plot(km_data, unique_target_models)
#km_bokeh_plot(km_data[6:12], unique_target_models[6:12])

In [17]:

def basic_km_plot(km, model):
    s = km.survival_function_
    ci = km.confidence_interval_
    time = s['KM_estimate'].index
    surv = s['KM_estimate'].values
    surv_upper = ci['KM_estimate_upper_0.95'].values
    surv_lower = ci['KM_estimate_lower_0.95'].values
    band_x = np.append(time, time[::-1])
    band_y = np.append(surv_upper, surv_lower[::-1])
    
    source = ColumnDataSource(
        data=dict(
            stime=[i for i in time],
            ssurv=[i for i in surv],
            model=[model for i in time],
            ssurv_lower=[i for i in surv_lower],
            ssurv_upper=[i for i in surv_upper]
        )
    )
    surv_plt1 = figure(title="Survival Analysis", tools=['box_zoom,save,reset'])
    surv_plt1.patch(band_x, band_y, color='#7570B3', fill_alpha=0.2)
    sline = surv_plt1.line('stime', 'ssurv', line_width = 4, source = source, legend=model, color='navy')#,  text_font_size ='12pt')

    show(surv_plt1)
         
model = 'alpha'
basic_km_plot(km, model)

In [92]:
from time import localtime, strftime, gmttime
THE_TIME = strftime("%Y-%m-%d-%H-%M", time())
print THE_TIME

ImportError: cannot import name gmttime

In [ ]:
#from lifelines import KaplanMeierFitter
from lifelines.datasets import load_waltons
df = load_waltons() # returns a Pandas DataFrame
T = df['T']
E = df['E']
kmf = KaplanMeierFitter()
kmf.fit(T, event_observed=E)
#plt.show()
km = sa.KaplanMeierFitter()
km.fit(T, event_observed=E)
#print km.data()
x = kmf.survival_function_
#kmf.survival_function_.plot()
ax = km.plot(title="Kaplan Meier fit", legend=False)
ax.annotate('half-life', color='black', xy=(0.01,0.5), xycoords='axes fraction', xytext=(10,4), textcoords='offset points')
ax.axhline(.5, ls='--', lw=1.0, color='black')
ax.set_ylim([0,1])
#ax.set_xlim([0,6])
print km.median_
plt.show()